In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from compute_distance import compute_distance

In [2]:
weights_path = 'models/periodic'

h_n = 64
input_n = 4
n_layers = 8

ds = 0.4
dp = 0.1

folder_path = "data/unscaled_p"
model = 'epoch_1000'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

In [5]:

# General PINN architecture for u, v, or p
class PINN(nn.Module):
    def __init__(self, input_dim=4, output_dim=1, hidden_dim=64, num_layers=8):
        super(PINN, self).__init__()
        layers = []

        # Input layer
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(Swish())

        # Hidden layers
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(Swish())

        # Output layer
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [6]:
class PeriodicYLayer(nn.Module):
    def __init__(self, m=16, n_features=64, L2=0.4, activation='tanh'):
        super().__init__()
        self.m = m
        self.n_features = n_features
        self.omega2 = 2.0 * np.pi / L2

        self.A1 = nn.Parameter(torch.randn(m))
        self.c1 = nn.Parameter(torch.zeros(m))

        self.A2   = nn.Parameter(torch.randn(m))
        self.phi2 = nn.Parameter(torch.zeros(m))
        self.c2   = nn.Parameter(torch.zeros(m))

        self.W1 = nn.Parameter(torch.randn(m, n_features) * (1.0 / np.sqrt(m)))
        self.W2 = nn.Parameter(torch.randn(m, n_features) * (1.0 / np.sqrt(m)))
        self.B  = nn.Parameter(torch.zeros(n_features))

        self.act = torch.tanh if activation == 'tanh' else Swish()

    def forward(self, x, y):
        if x.ndim == 1: x = x.unsqueeze(1)
        if y.ndim == 1: y = y.unsqueeze(1)

        v1 = self.act(self.A1.view(1, -1) * x + self.c1.view(1, -1))
        v2 = self.act(self.A2.view(1, -1) * torch.cos(self.omega2 * y + self.phi2.view(1, -1)) + self.c2.view(1, -1))

        q = torch.matmul(v1, self.W1) + torch.matmul(v2, self.W2) + self.B.view(1, -1)
        q = self.act(q)
        return q 

class PeriodicPINN(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=8, m_periodic=16, ds=0.4):
        super().__init__()
        assert num_layers >= 2, "num_layers counts all hidden layers including the periodic one."

        self.per_y = PeriodicYLayer(m=m_periodic, n_features=hidden_dim, L2=ds, activation='tanh')

        layers = []
        in_dim = hidden_dim + 2 
        for _ in range(num_layers - 2): 
            layers += [nn.Linear(in_dim, hidden_dim), Swish()]
            in_dim = hidden_dim
        self.body = nn.Sequential(*layers)

        self.out = nn.Linear(in_dim, 1)

        self.apply(self._init_xavier)

    @staticmethod
    def _init_xavier(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            nn.init.zeros_(m.bias)

    def forward(self, inp):
        x = inp[:, 0:1]
        y = inp[:, 1:1+1]
        d = inp[:, 2:3]
        n = inp[:, 3:4]

        q = self.per_y(x, y)     

        h = torch.cat([q, d, n], dim=1)
        h = self.body(h)
        return self.out(h)         
    

In [7]:
def initialize_models(model='best'):
    net_u = PINN(input_dim=input_n, output_dim=1, hidden_dim=h_n, num_layers=n_layers).to(device)
    net_v = PINN(input_dim=input_n, output_dim=1, hidden_dim=h_n, num_layers=n_layers).to(device)
    net_p = PINN(input_dim=input_n, output_dim=1, hidden_dim=h_n, num_layers=n_layers).to(device)
    
    # net_u = PeriodicPINN(hidden_dim=h_n, num_layers=n_layers, m_periodic=16, ds=ds).to(device)
    # net_v = PeriodicPINN(hidden_dim=h_n, num_layers=n_layers, m_periodic=16, ds=ds).to(device)
    # net_p = PeriodicPINN(hidden_dim=h_n, num_layers=n_layers, m_periodic=16, ds=ds).to(device)

    def init_xavier(model):
        for m in model.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)  # or small constant like 1e-3

    net_u.apply(init_xavier)
    net_v.apply(init_xavier)
    net_p.apply(init_xavier)

    net_u.load_state_dict(torch.load(f"{weights_path}/{model}_u.pth"))
    net_v.load_state_dict(torch.load(f"{weights_path}/{model}_v.pth"))
    net_p.load_state_dict(torch.load(f"{weights_path}/{model}_p.pth"))

    return net_u, net_v, net_p

In [8]:
net_u, net_v, net_p, *_ = initialize_models(model)

# Define value ranges
first_values = np.arange(0.25, 0.75, 0.05)  # 0.25 to 0.7 with step 0.05
# second_values = [10]  # 3 to 14
second_values = np.arange(3, 15, 1)  # 5 to 14

# Iterate over all possible files
for first in first_values:
    for second in second_values:
        first_str = f"{first:.2f}".rstrip("0").rstrip(".")
        file_name = f"TrP4_{first_str}_{second}.csv"
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            
            # Load CSV file
            df = pd.read_csv(file_path)
            
            # Extract input and output data
            input_data = df.iloc[:, [0, 1, 5, 7]].values  # (x, y, d, N) [0, 1, 5, 7]
            output_data = df.iloc[:, [2, 3, 4]].values  # (u, v, p)
            
            input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)
            x,y,d,n = input_tensor[:, 0], input_tensor[:, 1], input_tensor[:, 2], input_tensor[:, 3]
            
            # Predict using trained models
            with torch.no_grad():
                u_pred = net_u(input_tensor)
                v_pred = net_v(input_tensor)
                p_pred = net_p(input_tensor)

            # Apply conditions
            v_pred = v_pred.flatten()
            u_pred = u_pred.flatten()
            p_pred = p_pred.flatten()

            distances = compute_distance(x, y, d, n, ds)

            # xStart = 0
            # xEnd = ds

            # u_pred = u_pred * distances
            # v_pred = v_pred * distances

            # xStart = 0
            # xEnd = ds

            # p_par = ((xEnd - x) / (xEnd - xStart) * dp) + ((x - xStart) * (xEnd - xStart) * 0.0)
            # Dp = (x - xStart) * (xEnd - x)
            # p_pred = p_par + Dp * p_pred

            # u_pred = u_pred * distances
            # v_pred = v_pred * distances
            # p_pred = (xStart - x)*0 + dp * (xEnd - x) / ds + 0 * y + (xStart - x) * (xEnd - x) * p_pred

            inlet_condition = (torch.abs(x) < 1e-6)
            outlet_condition = (torch.abs(x - ds) < 1e-6)
            wall_condition = (distances < 1e-6)

            v_pred = torch.where(inlet_condition, torch.zeros_like(v_pred), v_pred)
            p_pred = torch.where(inlet_condition, torch.full_like(p_pred, dp), p_pred)

            p_pred = torch.where(outlet_condition, torch.zeros_like(p_pred), p_pred)

            u_pred = torch.where(wall_condition, torch.zeros_like(u_pred), u_pred)
            v_pred = torch.where(wall_condition, torch.zeros_like(v_pred), v_pred)

            # Prepare DataFrame for output
            df_output = pd.DataFrame({
                'x': input_data[:, 0],
                'y': input_data[:, 1],
                'u': u_pred.cpu().numpy(),
                'v': v_pred.cpu().numpy(),
                'p': p_pred.cpu().numpy()
            })
            
            # Save velocity output file
            output_file = f"pred_{first_str}_{second}.csv"
            output_path = os.path.join(fr"results\csv", output_file)
            df_output.to_csv(output_path, index=False)
            print(f"Saved {output_file} successfully!")

print("Processing complete!")


Processing TrP4_0.25_3.csv...
Saved pred_0.25_3.csv successfully!
Processing TrP4_0.25_4.csv...
Saved pred_0.25_4.csv successfully!
Processing TrP4_0.25_5.csv...
Saved pred_0.25_5.csv successfully!
Processing TrP4_0.25_6.csv...
Saved pred_0.25_6.csv successfully!
Processing TrP4_0.25_7.csv...
Saved pred_0.25_7.csv successfully!
Processing TrP4_0.25_8.csv...
Saved pred_0.25_8.csv successfully!
Processing TrP4_0.25_9.csv...
Saved pred_0.25_9.csv successfully!
Processing TrP4_0.25_10.csv...
Saved pred_0.25_10.csv successfully!
Processing TrP4_0.25_11.csv...
Saved pred_0.25_11.csv successfully!
Processing TrP4_0.25_12.csv...
Saved pred_0.25_12.csv successfully!
Processing TrP4_0.25_13.csv...
Saved pred_0.25_13.csv successfully!
Processing TrP4_0.25_14.csv...
Saved pred_0.25_14.csv successfully!
Processing TrP4_0.3_3.csv...
Saved pred_0.3_3.csv successfully!
Processing TrP4_0.3_4.csv...
Saved pred_0.3_4.csv successfully!
Processing TrP4_0.3_5.csv...
Saved pred_0.3_5.csv successfully!
Proces